## Cocomo Nasa V1

In [1]:
import numpy as np

import pandas as pd
import matplotlib.pyplot as plt

from scipy.io import arff

#algoritmos ia
from sklearn.neighbors import KNeighborsRegressor 
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import svm 

from sklearn import model_selection as ms
from sklearn.datasets import make_regression

## Preparando Dados

In [32]:
#Carregando dados

data, meta = arff.loadarff('datasets/cocomonasa_v1.arff')
df = pd.DataFrame(data)
df.head()

,RELY,DATA,CPLX,TIME,STOR,VIRT,TURN,ACAP,AEXP,PCAP,VEXP,LEXP,MODP,TOOL,SCED,LOC,ACT_EFFORT
0,b'Nominal',b'High',b'Very_High',b'Nominal',b'Nominal',b'Low',b'Nominal',b'High',b'Nominal',b'Very_High',b'Low',b'Nominal',b'High',b'Nominal',b'Low',70.0,278.0
1,b'Very_High',b'High',b'High',b'Very_High',b'Very_High',b'Nominal',b'Nominal',b'Very_High',b'Very_High',b'Very_High',b'Nominal',b'High',b'High',b'High',b'Low',227.0,1181.0
2,b'Nominal',b'High',b'High',b'Very_High',b'High',b'Low',b'High',b'High',b'Nominal',b'High',b'Low',b'High',b'High',b'Nominal',b'Low',177.9,1248.0
3,b'High',b'Low',b'High',b'Nominal',b'Nominal',b'Low',b'Low',b'Nominal',b'Nominal',b'Nominal',b'Nominal',b'High',b'High',b'Nominal',b'Low',115.8,480.0
4,b'High',b'Low',b'High',b'Nominal',b'Nominal',b'Low',b'Low',b'Nominal',b'Nominal',b'Nominal',b'Nominal',b'High',b'High',b'Nominal',b'Low',29.5,120.0


In [16]:
#removendo linhas com atributos nulos ou vazios

df = df.dropna()

#Convertendo atributos textuais em numeric com escala de atributos (ainda sem normalização) "OrdinalEncoder"

tranform_data = {b"Very_Low": 0, b"Low": 1, b"Nominal": 2, b"High": 3, b"Very_High": 4, b"Extra_High": 5}
df = df.replace(tranform_data)

#normalizando dados - #min-max df=(df-df.min())/(df.max()-df.min()) - #zscore df=(df-df.mean())/df.std()

df = (df-df.mean())/df.std()

#separando atributo classe Effort para entrada nos modelos #df.columns[-1]

X = df.drop(df.columns[-1], axis=1)
y = df.take([-1], axis=1)

## Treinando e avaliando o desempenho dos modelos

In [17]:
models = []
#Instanciando os modelos
models.append(['KNN', KNeighborsRegressor(n_neighbors=5)]) 
models.append(['MLP', MLPRegressor(activation='logistic', solver='sgd', alpha=0.02, max_iter=300, hidden_layer_sizes=200)])
models.append(['RFR', RandomForestRegressor()])
models.append(['SVR', svm.SVR()])

In [18]:
for model in models:
    cv = ms.KFold(n_splits=3, shuffle=True, random_state=1)
    scores = ms.cross_val_score(model[1], X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
    #converter para positivo.
    scores = np.absolute(scores)
    s_mean = np.mean(scores)
    s_std = np.std(scores)
    
    print('-', model[0],'------------------------------------')
    print('Scores:', scores, 'MAE: %.4f' % (s_mean), 'STD: %.4f' % (s_std)) 

- KNN ------------------------------------
Scores: [0.65105381 0.41409165 0.37898784] MAE: 0.4814 STD: 0.1208
- MLP ------------------------------------
Scores: [0.59592133 0.39459848 0.45705955] MAE: 0.4825 STD: 0.0841
- RFR ------------------------------------
Scores: [0.31391515 0.26760607 0.12394301] MAE: 0.2352 STD: 0.0809
- SVR ------------------------------------
Scores: [0.53730604 0.29982776 0.33528374] MAE: 0.3908 STD: 0.1046
